# **Image Captioning using ResNet-152 and LSTM**

## **Part 2: Training the Decoder LSTM for Caption Generation**

Once images have been encoded into feature vectors, a **Long Short-Term Memory (LSTM) network** is trained as a decoder to generate textual descriptions of the images.

### Steps:

1. **Initialize the LSTM Decoder**: The decoder takes the image feature vector (from the CNN encoder) as the initial input and processes sequential word embeddings to generate captions.
2. **Use an Embedding Layer**: Words in the vocabulary are mapped to dense vector representations to improve learning.
3. A **fully connected layer** maps the LSTM output to the vocabulary space.
4. **Optimize with Cross-Entropy Loss**: The decoder is optimized using a suitable loss function (categorical cross-entropy) to minimize the difference between predicted and actual words.
5. **Incorporate Teacher Forcing**: During training, the model uses actual words from the training set instead of its own predictions to improve learning stability.

The trained LSTM decoder enables the generation of fluent and contextually relevant captions for previously unseen images.

## **Technologies & Libraries**
- **Deep Learning Framework:** PyTorch and Tensforflow
- **Pretrained Model:** ResNet-152
- **Decoder Model**: custom LSTM-based decoder, developed in PyTorch
- **Text Processing:** Hugging Face Tokenizers
- **Dataset:** Flickr8k
- **Training Strategy:** Teacher Forcing

---

In [1]:
import sys
sys.path.insert(0,'../')

from tqdm import tqdm 
import torch
import pickle

from Pipeline.modelling.models.DecoderLSTM import *
from Pipeline.modelling.dataloader.Feature_Caption_Dataloader import *

### Load the encoded features and the captions

In [2]:
# load features
encoded_features = torch.load('./data/encoded_images_features.pt', weights_only=True)

# load captions
# Open the file in read-binary mode
with open("data/captions.pkl", "rb") as file:
    # Load the serialized list
    captions = pickle.load(file)

### Create a dataloader for the features

In [3]:
decoder_dataloader = Feature_Caption_Dataloader(encoded_features, captions, batch_size=64, shuffle=False)

### Create an instance of the Decoder

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
# use the same tokenizer used in the encoder
with open("data/custom_tokenizer.pkl", "rb") as file:
    # Load the serialized list
    tokenizer = pickle.load(file)

In [6]:
decoderLSTM = DecoderLSTM(
    vocab_size=tokenizer.my_vocabulary.__len__(),
    tokenizer=tokenizer
)

In [7]:
epoch_number = 30
optimiser = torch.optim.Adam(decoderLSTM.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

decoder = decoderLSTM.to(device)

for epoch in tqdm(range(epoch_number)):
    loss_value = 0
    
    for features, targets, lengths in decoder_dataloader:
        features = features.to(device)
        targets = targets.to(device)
        
        optimiser.zero_grad()
        outputs = decoder(features, targets, lengths)
        
        targets = pack_padded_sequence(targets, lengths, batch_first=True) # pack the targets the same way as the outputs
        loss_value = loss_function(outputs, targets.data)
        loss_value.backward()
        
        optimiser.step()
        
    if epoch % 1 == 0:
        print(f'Epoch {epoch} loss: {loss_value:.4f}')

  3%|▎         | 1/30 [00:09<04:21,  9.00s/it]

Epoch 0 loss: 2.9793


  7%|▋         | 2/30 [00:17<04:07,  8.83s/it]

Epoch 1 loss: 2.6482


 10%|█         | 3/30 [00:26<03:57,  8.80s/it]

...

Epoch 27 loss: 0.4532


 97%|█████████▋| 29/30 [04:13<00:08,  8.66s/it]

Epoch 28 loss: 0.4073


100%|██████████| 30/30 [04:22<00:00,  8.75s/it]

Epoch 29 loss: 0.3918


In [8]:
torch.save(decoder.state_dict(), 'data/trained_decoder.pt')

---